# Plotting using LSDMappingTools


In this tutorial we'll make some plots of the data generated using LSDTopoTools.  We'll firstly make a hillshade plot with overlaid channel networks, then some plots of the channel steepness analysis.

Firstly, import the required packages:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
rcParams['font.size'] = 14

# Now import some LSDMappingTools functions. This is some code that we generated 
# to do various raster plotting and analyses
import LSDPlottingTools.LSDMap_PointTools as LSDMap_PD
from LSDMapFigure.PlottingRaster import MapFigure

We're going to work with the data from the Pozo catchment, Santa Cruz Island. We need to define the name of the base raster (the DTM) and the hillshade:

In [ ]:
# Get the filenames
DataDirectory = '/home/fiona/PC_workshop/Pozo_test/'
BaseFile = 'Pozo_DTM'
RasterName = BaseFile+'.bil'
HSName = BaseFile+'_hs.bil'

## Plotting channel networks

We generated several CSV files during the previous stage of the workshop which have channel networks extracted through different methods. We can use the code below to plot these on top of the hillshade.

Select the channel network you want to plot. Simply change the extension depending on the channel extraction method: `_W_CN.csv` for the Wiener method, `_AT_CN.csv` for area threshold channels, `_D_CN.csv` for the integral method:

In [ ]:
ChannelName = BaseFile+'_W_CN.csv'

In [ ]:
# Create a map figure using the hillshade as the base raster
MF = MapFigure(HSName, DataDirectory,coord_type="UTM_km", colourbar_location='bottom',basemap_colourmap = "gray")

# Read in the channel network
ChannelPoints = LSDMap_PD.LSDMap_PointData(DataDirectory+ChannelName)
MF.add_point_data(ChannelPoints, scale_points = True,column_for_scaling = "Stream Order",
                    scaled_data_in_log = False,
                    max_point_size = 5, min_point_size = 1)

## Plotting chi profiles

We also ran some analysis to calculate channel steepness using the chi method across the landscape.  We can plot this too. Firstly, let's use `pandas` to load in the chi data:

In [ ]:
# get the dataframe with the chi data for the river channels
df = pd.read_csv(DataDirectory+BaseFile+'_MChiSegmented.csv')

In [ ]:
# Take a look at the column headers
print(df.columns)

Let's make some plots of the river profiles in the catchment. Let's group the dataframe by `source_key` (there is a unique source for each river) and then plot the elevation vs. distance from outlet:

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
gr = df.groupby('source_key')
gr.plot(x='flow_distance', y='elevation', ax=ax, legend=False)
ax.set_xlabel('Distance from outlet (m)')
ax.set_ylabel('Elevation (m)')

We can transform this into chi space to look at how steep the channels are. We plot the `chi` column on the X axis, `elevation` on the Y axis, and colour by the channel steepness (`m_chi`).

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
gr.plot(x='flow_distance', y='elevation', kind='scatter', s=1, c='m_chi', cmap=cm.viridis, ax=ax, legend=False, colorbar=False)
ax.set_xlabel('$\chi$ (m)')
ax.set_ylabel('Elevation (m)')